In [1]:
from eventregistry import *
import json, os, sys

from dotenv import load_dotenv
import os
from eventregistry import EventRegistry


load_dotenv()  # This loads the environment variables from .env file
api_key = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

In [2]:
conceptUri = "Tesla Cybertruck"
lang = "zho"
q = QueryArticlesIter(conceptUri=er.getConceptUri(conceptUri), lang=lang)

In [3]:
filename = conceptUri + "_" + lang + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(er, sortBy="date", maxItems=10):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, indent=4)

In [4]:
from openai import OpenAI

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def get_summary(
    art,
    client,
    model="gpt-3.5-turbo",
    language="English",
):

    prompt_complete = f"Summarize the following text in {language}: " + art["body"]
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [7]:
conceptUri = "Tesla Cybertruck"
lang = "zho"

src_filename = conceptUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in articles:
        src_art["summary_3.5_eng"] = get_summary(
            src_art, client, language="English", model="gpt-3.5-turbo"
        )
        src_art["summary_3.5_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-3.5-turbo"
        )
        src_art["summary_4o_eng"] = get_summary(
            src_art, client, language="English", model="gpt-4o"
        )
        src_art["summary_4o_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-4o"
        )
        des_articles.append(src_art)
        print(f'Summary in English(3.5): {src_art["summary_3.5_eng"]}')
        print(f'Summary in English(4o): {src_art["summary_4o_eng"]}')
        print(f'Summary in Chinese(3.5): {src_art["summary_3.5_zho"]}')
        print(f'Summary in Chinese(4o): {src_art["summary_4o_zho"]}')
        print(f'Original text: {src_art["body"]}')
        print("_____________________________")

des_filename = "summary" + "_" + conceptUri + "_" + lang + ".json"
if os.path.exists(des_filename):
    os.remove(des_filename)
with open(des_filename, "w") as f:
    json.dump(des_articles, f, indent=4)

Summary in English(3.5): According to the revised Article 6, Paragraph 3 of the "Computer Network Content Rating Processing Measures," websites are required to display ratings on their homepage or restricted pages in accordance with the regulations of the Taiwan Website Classification Promotion Fund. The Taiwan Website Classification Promotion Fund (TICRF) website is http://www.ticrf.org.tw. In entertainment news, popular YouTuber MrBeast celebrated his 26th birthday by giving away 26 Tesla cars, including 25 Model 3 cars and one Cybertruck. Participants can enter by liking and commenting on MrBeast's Instagram post and tagging two friends, with the giveaway limited to certain countries excluding Taiwan.
Summary in English(4o): According to the revised Article 6, Clause 3 of the "Regulations for the Classification of Computer Network Content," websites must display classification labels on their homepage or restricted pages as stipulated by the Taiwan Internet Content Rating Promotion 

In [6]:
conceptUri = "Tesla Cybertruck"
lang = "zho"

src_filename = conceptUri + "_" + lang + ".json"
with open(src_filename, "r") as f:
    data = f.read()
    des_articles = []
    src_articles = json.loads(data)
    for src_art in articles:
        src_art["summary_4o_eng"] = get_summary(
            src_art, client, language="English", model="gpt-4o"
        )
        src_art["summary_4o_zho"] = get_summary(
            src_art, client, language="Chinese", model="gpt-4o"
        )
        des_articles.append(src_art)
        print(f'Summary in English: {src_art["summary_4o_eng"]}')
        print(f'Summary in Chinese: {src_art["summary_4o_zho"]}')
        print(f'Original text: {src_art["body"]}')
        print("...............")

Summary in English: According to the revised Article 6, Clause 3 of the "Regulations on the Classification of Computer Network Content," websites must display classification labels on their homepage or restricted pages as per the Taiwan Internet Content Rating Promotion Foundation (TICRF) guidelines. 

In entertainment news, a popular YouTuber known as MrBeast, who has 245 million subscribers on YouTube, celebrated his 26th birthday by giving away 26 Tesla cars. This includes 25 Model 3 cars worth $39,000 each and one Cybertruck worth $120,000, totaling over 30 million NTD. To participate, fans need to like, comment, and tag two friends on his Instagram post. However, the giveaway is limited to residents of the USA, Australia, Canada, Japan, and South Korea, excluding Taiwan. The post has already garnered 12.7 million interactions.
Summary in Chinese: 根據「電腦網路內容分級處理辦法」修正條文第六條第三款規定，網站首頁或限制級網頁需依台灣網站分級推廣基金會規定作標示。網紅「Mr.R」在信義區撒10萬元引發騷動，效仿超級網紅「野獸先生（MrBeast）」。野獸先生在IG宣布將抽出26位幸運粉絲，每人可獲得一台特斯拉，總價值